In [19]:
import pickle
import random
import numpy as np
import cupy as cp
from unicode_info.database import generate_supported_consortium_feature_vectors_and_clusters_dict

In [15]:
unicode_codepoint_vectors_dict = pickle.load(open('features_dict_file.pkl', 'rb'))
unicode_index_codepoint_map = list(unicode_codepoint_vectors_dict.keys())
del unicode_codepoint_vectors_dict

unknown_codepoints_simvec_map = pickle.load(open('simmap_pointninethreshold.pkl', 'rb'))
mat = np.stack([vec for vec in unknown_codepoints_simvec_map.values()])
unknown_indices_to_codepoint = [codepoint for codepoint in unknown_codepoints_simvec_map.keys()]

In [62]:
# directly finding homoglyph codepoint sets will blow up memory
def filter_simvec_map_into_row_indices(sim_mat:np.ndarray, threshold:float):
    return set(cp.asnumpy(cp.nonzero(cp.count_nonzero(cp.array(sim_mat) > threshold, axis=1) - 1)[0]))

# these aren't equivalence classes - might be overlapping sets
def randomly_select_indices_and_find_homoglyphs_sets(sim_mat:np.ndarray, simmat_row_indices:set, threshold:float, num_sets:int, full_index_codepoint_map:list):
    pairs_for_index = {}
    sample = random.sample(simmat_row_indices, k=num_sets)
    print(sample)
    for row_index in sample:
        homoglyph_indices = np.nonzero(sim_mat[row_index] > threshold)[0]
        if len(homoglyph_indices) > 0:
            pairs_for_index[row_index] = {full_index_codepoint_map[index] for index in homoglyph_indices}
    return pairs_for_index

In [69]:
thres = 0.92
indices = filter_simvec_map_into_row_indices(mat, thres)
print(len(indices))
pairs_for_index = randomly_select_indices_and_find_homoglyphs_sets(mat, indices, thres, 5, unicode_index_codepoint_map)
print(len(pairs_for_index))
for s in pairs_for_index.values():
    print(len(s))

11775
[4147, 18301, 17177, 12271, 1941]
5
2
3
2
4
2


In [68]:
# debugging purpose
i = 18816
print(list(unknown_codepoints_simvec_map.items())[i])
print([unicode_index_codepoint_map[ind] for ind in np.nonzero(mat[i] > 0.92)[0]])

('8850', array([ 0.4917  ,  0.07587 ,  0.575   , ..., -0.10236 , -0.010994,
        0.3809  ], dtype=float16))
['8849', '8850']
